# Identificar o que é jogo de tiro e jogo de futebol

## Autor

- Levi de Oliveira Queiroz
- Matrícula: 170108341
- GitHub: LeviQ27

## Objetivo

O propósito do presente artigo é aplicar os conhecimentos adquiridos na lição 1 *Getting Started* do **Fast.ai** e montar um modelo de deep learning que identifica imagens de jogos de tiro e jogos de futebol

## Preparação

Antes de ser iniciado o passo a passo da identificação, é preciso instalar algumas dependencias como o *DuckDuckGo* *search* e o *fastai*.

In [ ]:
%pip install -U duckduckgo_search
%pip install -U fastai

Certo, com as dependências instaladas, vai ser iniciado o passo a passo da construção e aplicação do modelo de deep learning.

## Passo 1: Baixar as imagens de jogos de tiro e futebol

Para o primeiro passo, foi escrito o código que serviŕa de instalador das imagens usando o *ddg_images* do *DuckDcukGo*:

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

Para testar se as fotos foram recebidas, vai ser recebido uma URL de uma image de jogo de tiro pela função de pesquisa:

In [ ]:
urls = search_images("Shooting games", max_images=1)
urls[0]

Agora será feito o download da imagem pela URL:

In [ ]:
from fastdownload import download_url
dest = 'shooting_game.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Será realizado o download também para a foto do jogo de futebol:

In [ ]:
download_url(search_images('soccer videogame images', max_images=1)[0], 'soccer_game.jpg', show_progress=False)
Image.open('soccer_game.jpg').to_thumb(256,256)

Com isso, foi observado que a função *search_images* está entregando o retorno esperado do que foi aplicado para fazer. Assim, é trazido uma pequena parcela de fotos de tanto de jogos de tiro como de futebol e salvo cada uma em diferentes pastas de arquivo. Uma obesrvação é que é usado a função *sleep* para não haver dificuldades no tráfego de dados:

In [ ]:
searches = 'shooting game', 'soccer videogame'
path = Path('shooting_or_soccer')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} image'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} 2006 image'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} 2010 image'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

Como as funções anteriores estão apresentando o funcionamento esperado para elas, é dado sequência no paso a passo da construção do modelo.

## Passo 2: Treinando o Modelo

Nesse passo será feito o treino do modelo para realizar a tarefa de identificar qual imagem é um jogo de tiro e um jogo de futebol. 

Na sequência, é sabio perceber que algumas imagens não são baixadas corretamente, o que pode ocasionar na falha do treino do modelo, então com o seguinte código é removido essas imagens mal baixadas:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Feito isso, será treinado o modelo de deep learning. Para tal, é preciso do *DataLoaders*, que é um objeto que contém uma "sequência de treino", que são as imagens usadas para criar um modelo, e uma "sequência de validação", que são as imagens usadas para checar o quão certo é o modelo não usado durante o treino. Usando o *fastai* é possível criar esse treino de maneira fácil, consumindo pouca memória, usando a função *DataBlock*, que mostra também exemplos de imagen retornadas da função:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=12)

Agora, todos os preparatios estão feitos para treino o modelo de deep learning. A arquitetura de conecção de computação mais utilizado é o *resnet18* que é o que vai ser utilizado como base de aprendizado do modelo. O *fastai* vem com uma função *fine_tune()* que automaticamente usa as boas-práticas de afiar um modelo pré-treinado, o que é bastante útil.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Com isso, é seguido para o terceiro e último passo.

## Passo 3: Uso do Modelo

Com o modelo treinado, será testado como o modelo pensa sobre a imagem do jogo de tiro que foi baixado no início deste presente artigo:

In [ ]:
is_shooting_game_image,_,probs = learn.predict(PILImage.create('shooting_game.jpg'))
print(f"This is a: {is_shooting_game_image}.")
print(f"Probability it's a shooting game image: {probs[0]:.4f}")

## Conclusão

Nessa lição foi mostrado como que o uso de modelos de aprendizagem de máquina alterou o cotidiano das pessoas, como tarefas e atividades se tornaram triviais e fáceis de serem resolvidas por uma máuqina ou um sistema. A lição introduz o aluno no universo que é a aprendizagem de máquina, iniciando pelo modelo de identificação de imagens, porém esse modelo também serve para outras funcionalidades, essa abragência move a criatividade daquele que está aprendendo a como modelar e assim, construir modelos que vão transformar atividades que seriam complicadas para máquina e para o humano em tarefas triviais. Foi apresentado ao aluno ferramentas que facilitam o aprendizado e a construção dos modelos, como o notebook Jupyter com linguagem Python, utilizando de ferramentas que deixam o processo de treino de modelo eficiente e simples.